In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import sys
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install salesforce-lavis


Running in Colab.


In [ ]:

import torch
from PIL import Image
import os
from lavis.models import load_model_and_preprocess


In [ ]:

# Function to clear GPU memory
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

# Clear GPU memory before loading the model
clear_gpu_memory()


In [ ]:

# Setup device to use
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"


In [ ]:

# Load pretrained/finetuned BLIP2 captioning model
# Use a smaller model variant if available
try:
    model, vis_processors, _ = load_model_and_preprocess(
        name="blip2_t5", model_type="pretrain_flant5xl", is_eval=True, device=device
    )
except RuntimeError as e:
    print(f"Failed to load model on GPU: {e}. Switching to CPU.")
    device = torch.device("cpu")
    model, vis_processors, _ = load_model_and_preprocess(
        name="blip2_t5", model_type="pretrain_flant5xl", is_eval=True, device=device
    )

print(vis_processors.keys())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.89G/1.89G [00:08<00:00, 244MB/s]


...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

100%|██████████| 407M/407M [00:02<00:00, 175MB/s]


dict_keys(['train', 'eval'])


In [ ]:

#Directory containing the images
image_dir = '/content/drive/MyDrive/tile_dataset2'  #image folder path
output_dir = '/content/drive/MyDrive/output/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


#Iterate through each image file
for img_file in image_files:
    img_path = os.path.join(image_dir, img_file)
    try:
        raw_image = Image.open(img_path).convert('RGB')

        # Downscale the image to reduce memory usage
        raw_image = raw_image.resize((raw_image.width // 2, raw_image.height // 2))
        image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

        #Generate caption using beam search
        caption = model.generate({"image": image})
        print(f"Generated caption for {img_path} using beam search:", caption)

        # Generate multiple captions using nucleus sampling
        captions = model.generate({"image": image}, use_nucleus_sampling=True, num_captions=3)
        print(f"Generated captions for {img_path} using nucleus sampling:", captions)


        # Save the image and captions
        base_name = os.path.basename(img_path)
        raw_image.save(os.path.join(output_dir, base_name))

        with open(os.path.join(output_dir, base_name + '.txt'), 'w') as f:
            f.write("Caption using beam search:\n")
            f.write(caption[0] + "\n\n")
            f.write("Captions using nucleus sampling:\n")
            for c in captions:
                f.write(c + "\n")

    except Exception as e:
        print(f"Error processing {img_path}: {e}")

print("Processing completed. Results saved to:", output_dir)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Generated caption for /content/drive/MyDrive/tile_dataset2/Rocky_Mountains_USA_10_211_390.png using beam search: ['a map showing the location of a river in california']
Generated captions for /content/drive/MyDrive/tile_dataset2/Rocky_Mountains_USA_10_211_390.png using nucleus sampling: ['a map showing the location of a river in california', 'a map showing the location of a river', 'a map of the california national forest']
Generated caption for /content/drive/MyDrive/tile_dataset2/Rocky_Mountains_USA_10_211_389.png using beam search: ['a map showing the location of a hiking trail']
Generated captions for /content/drive/MyDrive/tile_dataset2/Rocky_Mountains_USA_10_211_389.png using nucleus sampling: ['a map showing the location of a hiking trail', 'a map showing the location of a hiking trail', 'a map showing the location of a hiking trail']
Generated caption for /content/drive/MyDrive/tile_dataset2/Rocky_Mountains_USA_10_212_391.png using beam search: ['a map showing the location of a

In [ ]:
import os
import json
import spacy

# Function to extract landmark entities from a caption using keyword matching
def extract_landmarks(caption):
    # List of landmarks to search for in the caption
    landmarks_list = ['mountain', 'river', 'lake', 'ocean', 'desert', 'forest','tree','trail', 'city', 'road']
    landmarks = []

    # Check for each landmark in the caption
    for landmark in landmarks_list:
        if landmark in caption.lower():
            landmarks.append(landmark)

    return landmarks

# Path to your tile dataset
dataset_path = '/content/drive/MyDrive/tile_dataset2'

# Initialize a list to store data for each tile
tiles_data = []

# Directory where captions are saved
output_dir = '/content/drive/MyDrive/output/'

# Iterate through each file in the dataset folder
for filename in os.listdir(dataset_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load caption file
        caption_file = os.path.join(output_dir, filename + '.txt')

        if os.path.exists(caption_file):
            with open(caption_file, 'r') as f:
                caption = f.read().strip()

                # Extract landmarks from caption
                landmarks = extract_landmarks(caption)

                # Prepare data for the current tile
                tile_data = {
                    'tile_name': filename,
                    'caption': caption,
                    'landmarks': landmarks
                }

                # Append the current tile's data to the list
                tiles_data.append(tile_data)
        else:
            print(f"Caption file not found for {filename}")

# Path to save the JSON file
output_json = '/content/drive/MyDrive/tiles_output/tiles_data.json'

# Save tiles_data to a JSON file
if not os.path.exists(os.path.dirname(output_json)):
    os.makedirs(os.path.dirname(output_json))

with open(output_json, 'w') as f:
    json.dump(tiles_data, f, indent=4)

print(f"JSON file saved successfully at: {output_json}")

JSON file saved successfully at: /content/drive/MyDrive/tiles_output/tiles_data.json


In [ ]:
import json

# Path to your JSON file
json_file_path = '/content/drive/MyDrive/tiles_output/tiles_data.json'

# Open and read the JSON file
with open(json_file_path, 'r') as f:
    tiles_data = json.load(f)

# Display the contents of the JSON file
print(json.dumps(tiles_data, indent=4))


[
    {
        "tile_name": "Rocky_Mountains_USA_10_211_390.png",
        "caption": "Caption using beam search:\na map showing the location of a river in california\n\nCaptions using nucleus sampling:\na map showing the location of a river in california\na map showing the location of a river\na map of the california national forest",
        "landmarks": [
            "river",
            "forest"
        ]
    },
    {
        "tile_name": "Rocky_Mountains_USA_10_211_389.png",
        "caption": "Caption using beam search:\na map showing the location of a hiking trail\n\nCaptions using nucleus sampling:\na map showing the location of a hiking trail\na map showing the location of a hiking trail\na map showing the location of a hiking trail",
        "landmarks": [
            "trail"
        ]
    },
    {
        "tile_name": "Rocky_Mountains_USA_10_212_391.png",
        "caption": "Caption using beam search:\na map showing the location of a lake\n\nCaptions using nucleus sampling:\